In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [103]:
df_enc = pd.read_csv("data/enc/data-enc-2024-05-27.csv")

In [104]:
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171336 entries, 0 to 171335
Data columns (total 86 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   avg_price_5                       171336 non-null  float64
 1   bathrooms_total                   171336 non-null  float64
 2   bedrooms_extra                    171336 non-null  float64
 3   bedrooms                          171336 non-null  float64
 4   stories_total                     171336 non-null  float64
 5   size_interior                     171336 non-null  float64
 6   lng                               171336 non-null  float64
 7   lat                               171336 non-null  float64
 8   parkings                          171336 non-null  float64
 9   price                             171336 non-null  float64
 10  household_income                  171336 non-null  float64
 11  individual_income                 171336 non-null  f

In [105]:
df_ON = df_enc[df_enc["province_Ontario"] == True]
df_QC = df_enc[df_enc["province_Quebec"] == True]
df_BC = df_enc[df_enc["province_British Columbia"] == True]
df_AB = df_enc[df_enc["province_Alberta"] == True]
df_SK = df_enc[df_enc["province_Saskatchewan"] == True]
df_MB = df_enc[df_enc["province_Manitoba"] == True]
# East
df_ES = df_enc[(df_enc["province_Nova Scotia"] == True) |
               (df_enc["province_New Brunswick"] == True) |
               (df_enc["province_Newfoundland & Labrador"] == True) |
               (df_enc["province_Prince Edward Island"] == True)]
# North
df_NO = df_enc[(df_enc["province_Yukon"] == True) |
               (df_enc["province_Northwest Territories"] == True)]

df_ON = df_ON.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_QC = df_QC.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_BC = df_BC.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_AB = df_AB.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_SK = df_SK.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_MB = df_MB.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_ES = df_ES.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta",
                    "province_Saskatchewan","province_Manitoba", "province_Yukon", "province_Northwest Territories"], axis=1)
df_NO = df_NO.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta",
                    "province_Saskatchewan", "province_Manitoba", "province_Nova Scotia", "province_New Brunswick",
                    "province_Newfoundland & Labrador", "province_Prince Edward Island"], axis=1)

In [106]:
df_AB.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23587 entries, 39041 to 171330
Data columns (total 74 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   avg_price_5                    23587 non-null  float64
 1   bathrooms_total                23587 non-null  float64
 2   bedrooms_extra                 23587 non-null  float64
 3   bedrooms                       23587 non-null  float64
 4   stories_total                  23587 non-null  float64
 5   size_interior                  23587 non-null  float64
 6   lng                            23587 non-null  float64
 7   lat                            23587 non-null  float64
 8   parkings                       23587 non-null  float64
 9   price                          23587 non-null  float64
 10  household_income               23587 non-null  float64
 11  individual_income              23587 non-null  float64
 12  commute_transit                23587 non-null 

In [107]:
df_AB.corr(numeric_only=True)["price"].sort_values(ascending=False).head(20)

price                         1.000000
size_interior                 0.707710
avg_price_5                   0.694295
bathrooms_total               0.540227
household_income              0.473251
individual_income             0.473251
building_type_House           0.354519
parkings                      0.323479
ownership_type_group_ids_1    0.310424
single_family                 0.285012
edu_bachelor_degree           0.284893
edu_post_graduate_degree      0.266072
bedrooms                      0.264658
owners                        0.260641
bedrooms_extra                0.252263
stories_total                 0.248962
edu_university_certificate    0.229566
area_single_detached          0.214529
age_50_to_64                  0.152566
lang_en_and_fr                0.145237
Name: price, dtype: float64

## Raw Input

In [108]:
kf = KFold(n_splits=10)

In [109]:
cb1 = CatBoostRegressor(silent=True)

X = df_AB.drop("price", axis=1)
y = df_AB["price"]

scores1 = cross_val_score(cb1, X, y, cv=kf, scoring="r2")

print(f"\n{scores1}\n")
pd.Series(scores1).describe()


[0.74681106 0.6094815  0.59086206 0.80245055 0.83692421 0.77998699
 0.82040902 0.86193215 0.7993114  0.87586272]



count    10.000000
mean      0.772403
std       0.098334
min       0.590862
25%       0.755105
50%       0.800881
75%       0.832795
max       0.875863
dtype: float64

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cb1.fit(X_train, y_train)
y_pred = cb1.predict(X_test)

In [111]:
r2_score(y_test, y_pred)

0.71852646481039

In [112]:
root_mean_squared_error(y_test, y_pred)

189604.3239677258

In [113]:
y_df1 = pd.DataFrame({"y_test":y_test.values, "y_pred":y_pred})

In [114]:
np.round(y_df1.head(20))

,y_test,y_pred
0,759900.0,852047.0
1,319900.0,303592.0
2,189900.0,210265.0
3,659000.0,654086.0
4,718377.0,604342.0
5,1099000.0,1054395.0
6,669900.0,806595.0
7,649900.0,645280.0
8,569900.0,533209.0
9,464900.0,440116.0


## Scaled

In [115]:
cb2 = CatBoostRegressor(silent=True)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1,1))

scores2 = cross_val_score(cb2, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores2}\n")
pd.Series(scores2).describe()

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb2.fit(X_train, y_train)
y_pred = cb2.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

0.5912273953202107

In [ ]:
root_mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred.reshape(-1,1)))

172787.8755555106

In [ ]:
y_df2 = pd.DataFrame({"y_test":scaler_y.inverse_transform(y_test[:,0].reshape(-1,1))[:,0].tolist(),
                      "y_pred":scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist()})

In [ ]:
np.round(y_df2.head(20))

,y_test,y_pred
0,189500.0,190314.0
1,295000.0,344583.0
2,349900.0,328338.0
3,236999.0,236757.0
4,229900.0,327002.0
5,399900.0,355567.0
6,205000.0,292441.0
7,319900.0,496840.0
8,199000.0,402424.0
9,779000.0,1100122.0


## log1p: X Only

In [ ]:
from scipy import stats

sub_df = df_AB.copy(deep=True)

skew_df = pd.DataFrame(sub_df.columns, columns=["Feature"])
skew_df["Corr_Price_Before"] = sub_df.corr()["price"].values
skew_df["Skew_Before"] = skew_df["Feature"].apply(lambda feature: stats.skew(sub_df[feature]))
skew_df["AbsSkew_Before"] = skew_df["Skew_Before"].apply(abs)
skew_df["Skewed_Before"] = skew_df["AbsSkew_Before"].apply(lambda x: True if x>= 0.5 else False)

# before_log = set(skew_df.query("Skewed_Before == True")["Feature"].values)

for column in skew_df.query("Skewed_Before == True")["Feature"].values:
    if column != "price":
        sub_df[column] = np.log1p(sub_df[column])


# skew_df = pd.DataFrame(sub_df.columns, columns=["Feature"])
skew_df["Corr_Price_After"] = sub_df.corr()["price"].values
skew_df["Skew_After"] = skew_df["Feature"].apply(lambda feature: stats.skew(sub_df[feature]))
skew_df["AbsSkew_After"] = skew_df["Skew_After"].apply(abs)
skew_df["Skewed_After"] = skew_df["AbsSkew_After"].apply(lambda x: True if x>= 0.5 else False)

# after_log = set(skew_df.query("Skewed_Before == True")["Feature"].values)

# log_columns = list(before_log - after_log - {"price", "lng", "lat"})
log_columns = skew_df[(skew_df["Skewed_Before"]==True) & (skew_df["Corr_Price_After"] > skew_df["Corr_Price_Before"])]["Feature"].values

log_columns

/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


array(['bedrooms_extra', 'commute_transit', 'commute_drive',
       'lang_en_only', 'lang_en_and_fr', 'edu_post_graduate_degree',
       'area_apt_5_plus_floors', 'loc_wellness', 'loc_greenery',
       'building_type_Row / Townhouse', 'building_type_Semi-Detached',
       'ownership_type_group_ids_0'], dtype=object)

In [ ]:
cb3 = CatBoostRegressor(silent=True)

X_log = X.copy(deep=True)
X_log[log_columns] = np.log1p(X_log[log_columns])

X_scaled = scaler_X.fit_transform(X_log)
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1,1))

scores3 = cross_val_score(cb3, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores3}\n")
pd.Series(scores3).describe()

KeyboardInterrupt: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb3.fit(X_train, y_train)
y_pred = cb3.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

0.7414105632911907

In [ ]:
root_mean_squared_error(y_test, y_pred)

0.5227749383645397

In [ ]:
y_df3 = pd.DataFrame({"y_test":scaler_y.inverse_transform(y_test[:,0].reshape(-1,1))[:,0].tolist(),
                      "y_pred":scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist()})

In [ ]:
np.round(y_df3.head(20))

,y_test,y_pred
0,1299000.0,1372134.0
1,559900.0,434518.0
2,369900.0,376609.0
3,939900.0,894375.0
4,699900.0,819007.0
5,790000.0,854122.0
6,629900.0,634359.0
7,275000.0,351898.0
8,599900.0,658089.0
9,1289900.0,1143188.0


# log1p: X and y

In [ ]:
cb4 = CatBoostRegressor(silent=True)

X_log = X.copy(deep=True)
X_log[log_columns] = np.log1p(X_log[log_columns])

y_log = np.log1p(y)

X_scaled = scaler_X.fit_transform(X_log)
y_scaled = scaler_y.fit_transform(y_log.to_numpy().reshape(-1,1))

scores4 = cross_val_score(cb4, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores4}\n")
pd.Series(scores4).describe()


[0.82182214 0.82146699 0.73593063 0.81733586 0.84662085 0.82513512
 0.82318958 0.8239074  0.82656587 0.82683272]



count    10.000000
mean      0.816881
std       0.029502
min       0.735931
25%       0.821556
50%       0.823548
75%       0.826208
max       0.846621
dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb4.fit(X_train, y_train)
y_pred = cb4.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

0.8277191904545071

### Reverse Log

In [ ]:
r2_score(np.expm1(y_test), np.expm1(y_pred))

0.34505682282317285

In [ ]:
root_mean_squared_error(y_test, y_pred)

0.4148795485106637

In [ ]:
y_df4 = pd.DataFrame({"y_test":np.expm1(scaler_y.inverse_transform(y_test))[:,0].tolist(),
                      "y_pred":np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist())})

In [ ]:
np.round(y_df4.head(20))

,y_test,y_pred
0,1299000.0,1377517.0
1,559900.0,454482.0
2,369900.0,367924.0
3,939900.0,913473.0
4,699900.0,811925.0
5,790000.0,824204.0
6,629900.0,606206.0
7,275000.0,288117.0
8,599900.0,574330.0
9,1289900.0,1130909.0


In [ ]:
np.round(y_df3.head(20))

,y_test,y_pred
0,1299000.0,1372134.0
1,559900.0,434518.0
2,369900.0,376609.0
3,939900.0,894375.0
4,699900.0,819007.0
5,790000.0,854122.0
6,629900.0,634359.0
7,275000.0,351898.0
8,599900.0,658089.0
9,1289900.0,1143188.0


In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_test)))
plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))

/tmp/ipykernel_3310/305300287.py:2: RuntimeWarning: overflow encountered in expm1
  plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_test)))
/tmp/ipykernel_3310/305300287.py:3: RuntimeWarning: overflow encountered in expm1
  plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))


/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/matplotlib/ticker.py:2096: RuntimeWarning: overflow encountered in multiply
  steps = self._extended_steps * scale
/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/matplotlib/ticker.py:2130: RuntimeWarning: overflow encountered in scalar subtract
  high = edge.ge(_vmax - best_vmin)
/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/matplotlib/ticker.py:1956: RuntimeWarning: invalid value encountered in scalar divmod
  d, m = divmod(x, self.step)


Error in callback <function _draw_all_if_interactive at 0x7f55fc250ea0> (for post_execute), with arguments args (),kwargs {}:


ValueError: arange: cannot compute length

ValueError: arange: cannot compute length

<Figure size 1200x800 with 1 Axes>

In [ ]:
r2_score(np.expm1(scaler_y.inverse_transform(y_test)), np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))

0.7199900915708094

In [ ]:
X_log

,avg_price_5,bathrooms_total,bedrooms_extra,bedrooms,stories_total,size_interior,lng,lat,parkings,household_income,...,loc_cafes,loc_quiet,loc_parks,building_type_Apartment,building_type_House,building_type_Row / Townhouse,building_type_Semi-Detached,ownership_type_group_ids_0,ownership_type_group_ids_1,ownership_type_group_ids_2
0,644900.0,2.0,0.000000,2.0,1.0,1072.000000,-83.029633,42.265783,0.0,125972.0819,...,1.5,4.5,3.5,0.693147,0.0,0.0,0.0,0.0,0.000000,1.0
1,1098759.8,3.0,0.000000,4.0,2.0,1965.881818,-83.087577,42.228649,0.0,205459.7539,...,1.0,5.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
2,347360.0,1.0,0.000000,4.0,1.0,1040.426146,-83.056441,42.304613,0.0,76724.5170,...,2.5,3.0,3.5,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
3,299537.6,1.0,0.000000,3.0,1.0,3718.455676,-83.055635,42.302468,0.0,78194.1950,...,2.0,4.0,3.5,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
4,895919.8,3.0,0.000000,4.0,2.0,2700.000000,-83.093660,42.134249,0.0,143383.8087,...,0.0,5.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154362,717920.0,3.0,0.000000,3.0,2.0,3092.004209,-77.505887,46.114368,4.0,159543.4167,...,0.0,5.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
154363,725900.0,2.0,0.693147,3.0,1.0,1386.740020,-77.516865,46.108780,5.0,152340.5896,...,0.0,4.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
154520,765740.0,1.0,0.000000,3.0,1.0,480.000000,-79.803065,47.059016,0.0,113000.0000,...,0.5,5.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0
154521,732940.0,0.0,0.000000,2.0,1.0,879.000000,-79.834607,47.047430,0.0,113000.0000,...,0.5,4.5,0.0,0.000000,1.0,0.0,0.0,0.0,0.693147,0.0


In [ ]:
y

0          399900.0
1          959900.0
2          299900.0
3          399000.0
4          999900.0
            ...    
154362    1100000.0
154363     465000.0
154520     385000.0
154521     549000.0
154896     394000.0
Name: price, Length: 77056, dtype: float64

In [ ]:
df_log = X_log.copy(deep=True)
df_log["price_log"] = np.log1p(y)

In [ ]:
df_log.corr(numeric_only=True)["price_log"].sort_values(ascending=False).head(20)

price_log                     1.000000
avg_price_5                   0.734018
bathrooms_total               0.620813
size_interior                 0.562767
household_income              0.473246
individual_income             0.473246
bedrooms                      0.470760
parkings                      0.424573
single_family                 0.360289
building_type_House           0.341128
ownership_type_group_ids_1    0.332655
stories_total                 0.294120
edu_bachelor_degree           0.293487
owners                        0.287807
edu_university_certificate    0.268325
bedrooms_extra                0.258855
edu_post_graduate_degree      0.233717
multi_family                  0.232745
age_15_to_19                  0.221740
age_10_to_14                  0.195383
Name: price_log, dtype: float64

In [ ]:
X.columns

Index(['avg_price_5', 'bathrooms_total', 'bedrooms_extra', 'bedrooms',
       'stories_total', 'size_interior', 'lng', 'lat', 'parkings',
       'household_income', 'individual_income', 'commute_transit',
       'commute_foot', 'commute_bicycle', 'commute_drive', 'single_family',
       'multi_family', 'single_person', 'multi_person', 'total_individuals',
       'age_0_to_4', 'age_5_to_9', 'age_10_to_14', 'age_15_to_19',
       'age_20_to_34', 'age_35_to_49', 'age_50_to_64', 'age_65_to_79',
       'age_80_plus', 'owners', 'renters', 'lang_en_only', 'lang_fr_only',
       'lang_en_and_fr', 'lang_other', 'edu_no_high_school', 'edu_high_school',
       'edu_trade_certificate', 'edu_college_certificate',
       'edu_university_certificate', 'edu_bachelor_degree',
       'edu_post_graduate_degree', 'household_children',
       'area_single_detached', 'area_semi_detached', 'area_duplex',
       'area_row_houses', 'area_apt_1_to_4_floors', 'area_apt_5_plus_floors',
       'loc_high_schools', 